<a href="https://colab.research.google.com/github/ollihansen90/VectorQuantisierung_Futureskills/blob/main/VecQuant_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kapitel 7: Programmierung des kMeans-Clustering
In diesem Notebook geht es um die Implementierung des kMeans-Algorithmus. Hierbei werden sowohl Pattern-by-Pattern- als auch Batch-Learning behandelt.

## Setup
Im Setup werden drei Punktewolken generiert und danach eingezeichnet.

In [ ]:
# TODO: Auf dem Jupyter-Hub wird die utils.py lokal gespeichert und muss nicht mit wget von Github gezogen werden.
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/VectorQuantisierung_Futureskills/main/utils.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import setup, zugehoerigkeit, draw, run_experiments

data, _ = setup()

plt.figure()
plt.scatter(data[:,0], data[:,1])
plt.show()


## Pattern-by-Pattern-Learning
Beim Pattern-by-Pattern-Learning werden die Datenpunkte durchiteriert und der jeweils nächste Codebookvektor leicht verändert. Die Idee dabei ist sehr ähnlich zum Berechnen des Mittelwerts über das exponentielle Mittel, wie es im Notebook zu Kapitel 3 beschrieben wurde.

Im folgenden Codeblock wird der kMeans-Update für Pattern-by-Pattern-Learning implementiert. Für einen beliebigen Punkt aus der Menge der Datenpunkte wird zunächst der am nächsten liegende Codebookvektor gesucht. Im Anschluss wird der Codebookvektor aktualisiert. Eine Lernrate von 0.1 bedeutet hier, dass der Codebookvektor um 10% der Strecke in Richtung gegebenen Punkt verschoben wird.

In [ ]:
def kmeans_update_pbp(codebook, point, lernrate=0.1):
    cb = codebook.copy()
    # finde nächsten Codebookvektor
    dist = np.inf
    for i in range(len(cb)):
        abstand = np.sum((cb[i]-point)**2)**0.5 # Berechne Abstand (hier: euklidischer Abstand)
        if abstand<dist: # Wenn der aktuelle Abstand kürzer als der bisher kürzeste ist...
            dist = abstand
            naechster_index = i
    # Update für den nächsten Codebookvektor
    cb[naechster_index] = cb[naechster_index]+lernrate*(point-cb[naechster_index])
    return cb

Nachfolgend kann der Algorithmus für unterschiedliche $k$ getestet werden.

In [ ]:
# --- Hier kann das k angepasst werden ---
k = 8
# ----------------------------------------
codebook = np.random.rand(k, 2)*(np.max(data, axis=0)-np.min(data, axis=0))+np.min(data, axis=0)
label, error = zugehoerigkeit(codebook, data)
draw(codebook, data, title="Initialisierung")
print(f"Fehler bei Initialisierung: {error}")

for i in range(3):
    for p in np.random.permutation(data):
        codebook = kmeans_update_pbp(codebook, p, lernrate=0.1)
    label, error = zugehoerigkeit(codebook, data)
    draw(codebook, data, title=f"Epoche {i+1}")
    print(f"Fehler in Epoche {i+1}: {error}")


## Batch-Learning
Das Batch-Learning liefert genau den Updateschritt, wie er in den Videos beschrieben wird: Jeder Codebookvektor wird auf den Schwerpunkt seiner Voronoi-Menge verschoben.

Im folgenden Codeblock wird der kMeans-Update für Batch-Learning implementiert. Zunächst werden die Punkte auf die Voronoi-Mengen der jeweiligen Codebookvektor verteilt. Im Anschluss wird der Codebookvektor aktualisiert, indem er auf den Schwerpunkt seiner Voronoi-Menge verschoben wird.

In [ ]:
def kmeans_update_b(codebook, data):
    cb = codebook.copy()

    # finde nächsten Codebookvektor
    voronoi = np.zeros(len(data))
    for i in range(len(data)):
        dist = np.inf
        for j in range(len(cb)):
            abstand = np.sum((cb[j]-data[i])**2)**0.5 # Euklidischer Abstand
            if abstand<dist:
                dist = abstand
                voronoi[i] = j

    # Update der Codebookvektoren
    for i in range(len(cb)):
        # Setze cb auf den Schwerpunkt seiner Voronoi-Menge
        voronoimenge = np.arange(len(data))[voronoi==i]
        if len(voronoimenge)>0:
            # Achtung: Wenn die Voronoi-Menge leer ist, produziert Python hier einen Fehler
            cb[i] = np.mean(data[voronoimenge], axis=0)
    return cb

Nachfolgend kann der Algorithmus für unterschiedliche $k$ getestet werden.

In [ ]:
# --- Hier kann das k angepasst werden ---
k = 8
# ----------------------------------------
codebook = np.random.rand(k, 2)*(np.max(data, axis=0)-np.min(data, axis=0))+np.min(data, axis=0)
label, error = zugehoerigkeit(codebook, data)
draw(codebook, data, title="Initialisierung")
print(f"Fehler bei Initialisierung: {error}")

for _ in range(3):
    codebook = kmeans_update_b(codebook, data)
    _, error = zugehoerigkeit(codebook, data)
    draw(codebook, data, title=f"Epoche {i+1}")
    print(f"Fehler in Epoche {i+1}: {error}")

## Wahl für $k$
Zum Schluss stellt sich nun noch die Frage zur Wahl der besten Anzahl an Codebookvektoren im Codebook. Es gibt unterschiedliche andere Ansätze, die dynamisch die Anzahl $k$ im Verlauf der Anpassung bestimmen (z. B. DBSCAN oder OPTICS), der klassische kMeans-Algorithmus muss jedoch vorher genau wissen, wie viele Codebookvektoren existieren.

Der einfachste Ansatz wäre es, wenn man sich die Daten einmal anzeigen lässt und dann die Anzahl der Cluster durchzählt. Das funktioniert jedoch leider nur bei sehr niedrigdimensionalen Daten! Bei zwei Dimensionen (und manchmal bei drei) kann das oft durch einen Blick bestimmt werden, doch wie soll das bei zehn Dimensionen gehen? Offensichtlich können zehndimensionale Daten nicht so einfach auf einem zweidimensionalen Bildschirm dargestellt werden.

Der Vorgang ist jedoch denkbar einfach: kMeans wird mit unterschiedlichen Werten $k$ initialisiert und danach an die Daten angepasst, bis die Codebookvektoren sich nicht oder nur sehr wenig verändern. Man sagt auch: Das Verfahren *konvergiert*. Zum Schluss werden die Fehler einzelner $k$ verglichen und entschieden, für welches $k$ der Algorithmus die besten Ergebnisse liefert.

Im folgenden Codeblock wurde dieses Verfahren einmal implementiert. Bei den Daten handelt es sich um 180 zehndimensionale Punkte, die auf sechs Cluster aufgeteilt sind. Diese Cluster sind jeweils normalverteilt mit unterschiedlichen Mittelwerten und Standardabweichungen. Der kMeans-Algorithmus wird nun mit unterschiedlichen Werten für $k$ initialisiert. Hierbei wurden Werte zwischen 1 und 360 gewählt. Jedes Anpassen wurde fünf Mal mit unterschiedlichen Initialisierungen wiederholt, um eine möglichst gute Schätzung für den mittleren Fehler zu erhalten.

In [ ]:
d = 10
n = 30
datalist = []
for i in range(6):
    scale = 2*np.random.rand()+0.5
    offset = np.random.randn(d)
    offset /= np.linalg.norm(offset)
    data = np.random.randn(n, d)*scale+10*offset
    datalist.append(data)
data = np.row_stack(datalist)

run_experiments(data, n_runs=5)

Es ist gut zu erkennen, wie der Fehler mit steigendem $k$ kleiner wird. Für $k=1$ ist der Fehler vergleichsweise schlecht. Das ist auch nicht verwunderlich: Der kMeans-Algorithmus für den einen Codebookvektor auf den Mittelpunkt der Daten verschieben. Für größerwerdendes $k$ fällt der Fehler stark ab.

Es ist gut zu erkennen, dass nach $k=6$ das Abfallen etwas schwächer wird, wenn auch nicht ausbleibt. Das liegt in erster Linie daran, dass die gegebenen Daten aus sechs Clustern bestehen, die für $k=6$ jeweils einen Codebookvektor erhalten.

Ab $k=180$ ist der Fehler 0. Auch das ist nicht verwunderlich, denn für $k=180$ gibt es für jeden Datenpunkt einen Codebookvektor. Für $k>180$ gibt es sogar mehrfachbelegte Punkte.

Für die Wahl der besten Anzahl der Clusterzentren wird genau so eine Abbildung betrachtet. Sicherlich könnte direkt die Anzahl Punkte als Anzahl Codebookvektoren gewählt werden, nur wäre damit nichts gewonnen. Ein geeignetes $k$ sollte groß genug sein, dass der Fehler hinreichend klein ist, allerdings nicht so groß, dass der Speicheraufwand zu groß wird.

In dem Beispiel hier hieße, dass $k=100$ einen Fehler hat, der halb so groß wie der von $k=6$ ist, der Speicheraufwand ist jedoch mehr als 16 Mal so groß. Hier würden wir uns also für $k=6$ entscheiden.